In [1]:
from torch.utils.data import Dataset
import torch

data = torch.load("sarcasm_dataset.pt")
input_ids = data['input_ids']
attention_mask = data['attention_mask']
labels = data['labels']
texts = data['texts']

class SarcasmDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, texts):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.texts = texts

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
            'texts': self.texts[idx]
        }

texts = data['texts']
dataset = SarcasmDataset(input_ids, attention_mask, labels, texts)

In [ ]:
dataset_size = len(dataset)
print(f"Dataset size: {dataset_size}")

Dataset size: 1126324


In [3]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
import torch

SEED = 4213
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Extract numpy arrays for splitting
labels_np = labels.numpy()

# First split: train (80%) + temp (20%)
train_idx, temp_idx = train_test_split(
    np.arange(len(labels_np)),
    test_size=0.2,
    random_state=SEED,
    stratify=labels_np
)

# Second split: validation (10%) + test (10%) from temp
val_idx, test_idx = train_test_split(
    temp_idx,
    test_size=0.5,
    random_state=SEED,
    stratify=labels_np[temp_idx]
)

train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)

BATCH_SIZE = 32 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [4]:
def count_labels(dset, name):
    lbls = [dset[i]['labels'].item() for i in range(len(dset))]
    unique, counts = np.unique(lbls, return_counts=True)
    print(f"{name} distribution:", dict(zip(unique, counts)))

count_labels(train_dataset, "Train")
count_labels(val_dataset, "Validation")
count_labels(test_dataset, "Test")

Train distribution: {0: 443182, 1: 457877}
Validation distribution: {0: 55398, 1: 57234}
Test distribution: {0: 55398, 1: 57235}


In [ ]:
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
import torch.nn as nn

model_name = "bert-base-uncased"

# Load BERT with classification head
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Freeze ALL BERT parameters
for param in model.bert.parameters():
    param.requires_grad = False

# Now only classifier layer will train
print("Trainable parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.classifier.parameters(), lr=1e-4)  # Only classifier params
loss_fn = CrossEntropyLoss()

EPOCHS = 3

for epoch in range(EPOCHS):
    # ---- Training ----
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {avg_train_loss:.4f}, "
          f"Val Loss = {avg_val_loss:.4f}, "
          f"Val Acc = {val_accuracy:.4f}")
    
#Save the model
torch.save(model.state_dict(), "frozen_bert_model.pth")

In [ ]:
from transformers import BertForSequenceClassification
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load weights
state_dict = torch.load("frozen_bert_model.pth", map_location="cpu")
model.load_state_dict(state_dict)

model.eval()
print("✅ Model loaded successfully and ready for inference!")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Move model to the selected device
test_loss, correct, total = 0, 0, 0

# Calculate precision, recall, F1-score
from sklearn.metrics import precision_score, recall_score, f1_score
all_preds = []
all_labels = []
all_preds_prob = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        preds = outputs.logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        probs = torch.softmax(outputs.logits, dim=1)
        preds = probs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_preds_prob.extend(probs[:, 1].cpu().numpy())  # Probability of positive class

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Accuracy: {correct/total:.4f}")

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_preds_prob = np.array(all_preds_prob)

precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# --- Confusion Matrix ---
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Sarcastic", "Sarcastic"])
disp.plot(cmap="Blues", values_format='d')
plt.title("Confusion Matrix")
plt.show()

# --- ROC Curve and AUC ---
fpr, tpr, thresholds = roc_curve(all_labels, all_preds_prob)
roc_auc = roc_auc_score(all_labels, all_preds_prob)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.4f})")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"ROC AUC Score: {roc_auc:.4f}")

Test Loss: 0.6369, Accuracy: 0.6389

Precision: 0.6356, Recall: 0.6782, F1-Score: 0.6562

ROC AUC Score: 0.6949